In [314]:
import cadquery as cq
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show)
from jupyter_cadquery import set_sidecar, set_defaults

import alphashape as alph

import numpy as np
from scipy.ndimage import affine_transform

set_defaults(default_color="#ECF0FF", axes=False, grid=True,theme="light", axes0=True, ortho=True, transparent=True)
set_sidecar("CadQuery", init=True)


in2mm = lambda x: x*25.4

## Bottom Plate

In [354]:
widthmm = in2mm(14)
heightmm = in2mm(6.5)
depthmm = in2mm(1/8)
screwhole_diameter = 9

# base 
base = cq.Workplane('XY').box(widthmm, heightmm, depthmm).edges("|Z").fillet(5)

# Corner holes
offset = 50
bottom = base.faces(">Z").workplane().rect(widthmm-offset, heightmm-offset, forConstruction=True)\
                                     .vertices().hole(screwhole_diameter)

show(bottom) 

Done, using side car 'Cadquery'


In [343]:
base

Done, using side car 'Cadquery'


## Hollow Plate

In [344]:
hollow_offset_x = 80
hollow_offset_y = 80

cavity = cq.Workplane('XY').box(widthmm-hollow_offset_x, heightmm-hollow_offset_y, depthmm)
shell = base.faces(">Z").workplane().rect(widthmm-offset, heightmm-offset, forConstruction=True)\
                                     .vertices().hole(screwhole_diameter)

shell = shell.faces(">Z").workplane().rect(0, heightmm-offset, forConstruction=True).vertices().hole(screwhole_diameter)
shell = shell.cut(cavity)
show(shell)

Done, using side car 'Cadquery'


In [370]:
eightbutton_coords = [(-26.15, -11.75), (0,0), (28.65, -3.2), (53.5, -19.45), 
                      (-26.15, -33.75), (0, -22), (28.65, -25.2), (53.5, -41.45)]

# the four buttons for directionals is just the first 4 buttons reflected along the Y axis
fourbutton_coords = np.matmul(eightbutton_coords[4:], [[-1, 0], [0, 1]])

cstick_coords = [(0,0), (-19, -13.25), (0, 26.65), (-19, 13.25), (19, 13.25)]
# modifier buttons are just the first two cstick buttons reflected along the Y axis
modifier_coords = np.matmul(cstick_coords[:2], [[-1, 0], [0, 1]])
startbutton_coords = [(0,0)]

eightbutton_center = (100, 58)
fourbutton_center = (-100, 58)
modifier_center = (-70, -45)
cstick_center = (70, -45)
startbutton_center = (0, 18)

# for whatever reason, trying to cut everything at once causes an error,
# so we have to do it one at a time
eightbutton_layout = np.add(eightbutton_coords, eightbutton_center)  
fourbutton_layout = np.add(fourbutton_coords, fourbutton_center)
modifier_layout = np.add(modifier_coords, modifier_center)
cstick_layout = np.add(cstick_coords, cstick_center)
startbutton_layout = np.add(startbutton_coords, startbutton_center)

complete_layout = np.concatenate([eightbutton_layout, fourbutton_layout, modifier_layout, cstick_layout, startbutton_layout], axis=0)

cap_diameter = 11.5

f1_cap_faceplate = base.faces(">Z").pushPoints(fourbutton_layout).circle(cap_diameter).cutThruAll()
f1_cap_faceplate = f1_cap_faceplate.pushPoints(eightbutton_layout).circle(cap_diameter).cutThruAll()
f1_cap_faceplate = f1_cap_faceplate.pushPoints(modifier_layout).circle(cap_diameter).cutThruAll()
f1_cap_faceplate = f1_cap_faceplate.pushPoints(cstick_layout).circle(cap_diameter).cutThruAll()
f1_cap_faceplate = f1_cap_faceplate.pushPoints(startbutton_layout).circle(cap_diameter).cutThruAll()

f1_cap_faceplate

StdFail_NotDone: BRep_API: command not done

# Keeb Cap Faceplate

In [371]:
keebcap_dimensions = (19, 19)

keebcap_faceplate = base.faces(">Z").pushPoints(fourbutton_layout).rect(*keebcap_dimensions).cutThruAll()
keebcap_faceplate = keebcap_faceplate.pushPoints(eightbutton_layout).rect(*keebcap_dimensions).cutThruAll()
keebcap_faceplate = keebcap_faceplate.pushPoints(modifier_layout).rect(*keebcap_dimensions).cutThruAll()
keebcap_faceplate = keebcap_faceplate.pushPoints(cstick_layout).rect(*keebcap_dimensions).cutThruAll()
keebcap_faceplate = keebcap_faceplate.pushPoints(startbutton_layout).rect(*keebcap_dimensions).cutThruAll()

keebcap_faceplate

Done, using side car 'Cadquery'


In [372]:
# one with gaps removed
handcut_faceplate = base.faces(">Z").pushPoints(fourbutton_layout).rect(*keebcap_dimensions).cutThruAll()

# cut eight button cluster
eightbutton_dimensions = (19, 2*19+3)
eightbutton_center = np.add(eightbutton_layout[4:], eightbutton_layout[:4])/2
handcut_faceplate = handcut_faceplate.faces(">Z").pushPoints(eightbutton_center).rect(*eightbutton_dimensions).cutThruAll()

# cut modifiers
modifier_verts = cq.Workplane("XY").pushPoints(modifier_layout).rect(*keebcap_dimensions).vertices().vals()
coords_x = [v.X for v in modifier_verts]
coords_y = [v.Y for v in modifier_verts]
x,y = alph.alphashape(zip(coords_x, coords_y), alpha=0.0).boundary.xy
handcut_faceplate = handcut_faceplate.pushPoints(modifier_layout).rect(*keebcap_dimensions).cutThruAll()

# cut the start button
handcut_faceplate = handcut_faceplate.pushPoints(startbutton_layout).rect(*keebcap_dimensions).cutThruAll()
handcut_faceplate = handcut_faceplate.pushPoints(cstick_layout).rect(*keebcap_dimensions).cutThruAll()

# cut the cstick
cstick_verts = cq.Workplane("XY").pushPoints(cstick_layout).rect(*keebcap_dimensions).vertices().vals()
coords_x = [v.X for v in cstick_verts]
coords_y = [v.Y for v in cstick_verts]
x,y = alph.alphashape(zip(coords_x, coords_y), alpha=0.09).boundary.xy
handcut_faceplate = handcut_faceplate.polyline(list(zip(x,y))).close().cutThruAll()


handcut_faceplate

Done, using side car 'Cadquery'


# Switch Plate

In [373]:
switch_mount_dimensions = (14, 14)

switch_plate = base.faces(">Z").pushPoints(complete_layout).rect(*switch_mount_dimensions).cutThruAll()
switch_plate

Done, using side car 'Cadquery'


In [374]:
pico_width = 21
pico_height = 53

usbc_width = 19.5
usbc_height = 44
usbc_mount_width = 28
usbc_mount_height = 9

# cadquery is making getting the bbox coords hard for all the rects, so I am  going around it
vertices = base.faces(">Z").pushPoints(complete_layout).rect(*switch_mount_dimensions).vertices().vals()
coords_x = [v.X for v in vertices]
coords_y = [v.Y for v in vertices]
minx = min(coords_x)
maxx = max(coords_x)
miny = min(coords_y)
maxy = max(coords_y)
print(minx, maxx, miny, maxy)

usbc_slot = cq.Workplane('XY').hLine(usbc_mount_width/2).vLine(-usbc_mount_height).hLine(-(usbc_mount_width-usbc_width)/2).vLine(-1*(usbc_height-usbc_mount_height)).hLineTo(0).mirrorY()
pico_slot = cq.Workplane('XY').rect(pico_width, pico_height)

# extrude 
usbc_poly = usbc_slot.offset2D(1, kind="intersection").extrude(depthmm, both=True).translate((40, heightmm/2-1, 0))
cavity = cq.Workplane("XY").rect(maxx-minx, maxy-miny).extrude(depthmm, both=True).translate((0, -1.5))

wiring_cavity = base.cut(usbc_poly).cut(cavity)
show(wiring_cavity)

-160.5 160.5 -65.25 65.0
Done, using side car 'Cadquery'


In [375]:
screwhole_diameter = 9

# c = cq.Workplane('XY').polyline(np.transpose([clusterPolys.boundary.xy[0], clusterPolys.boundary.xy[1]])).edges().close().offset2D(4, kind='tangent').extrude(depthmm, both=True)
# pcb_holder_layer2 = pcb_holder.cut(c)
wiring_cavity_last_layer = wiring_cavity.faces(">Z").workplane().rect(widthmm-offset, heightmm-offset, forConstruction=True)\
                                     .vertices().hole(screwhole_diameter)

wiring_cavity_last_layer

Done, using side car 'Cadquery'


In [376]:
# type(verts.wire().val())
# verts = cq.Workplane("XY").add(verts.wire().val()).toPending().extrude(depthmm, both=True)

In [377]:
# show(pcb_holder.cut(verts))

In [378]:
cardboardmx = PartGroup(
    [
        # Part(cq.importers.importDXF('/mnt/chromeos/MyFiles/Downloads/3_SwitchPlate_v2.dxf').rotateAboutCenter((0,0,1), 180).translate((223.1, -181.25, 0)) ),
        Part(base, "bottom"),
        Part(bottom.translate((0,0,depthmm*1)), "shell"),
        Part(wiring_cavity_last_layer.translate((0,0,depthmm*2)), "pcb_holder2"),
        Part(wiring_cavity.translate((0,0,depthmm*3)), "pcb_holder"),
        Part(switch_plate.translate((0,0,depthmm*4)), "switchplate1"),
        Part(switch_plate.translate((0,0,depthmm*5)), "switchplate2"),
        Part(handcut_faceplate.translate((0,0,depthmm*6)), "faceplate1"),

    ],
    "carboardMX"
)

show(cardboardmx, default_color="blue")

Done, using side car 'Cadquery'


In [328]:
from cadquery import exporters

exporters.export(bottom, "artifacts/bottom.dxf")
exporters.export(wiring_cavity_last_layer, "artifacts/pcb_layer_1.dxf")
exporters.export(wiring_cavity, "artifacts/pcb_layer_2.dxf")
exporters.export(switch_plate, "artifacts/switch_plate.dxf")
exporters.export(keeb_cap_faceplate, "artifacts/keebcap_faceplate.dxf")
exporters.export(f1_cap_faceplate, "artifacts/f1_cap_faceplate.dxf")

In [42]:
# exporters.export(cardboardmx, "artifacts/controller.svg")